# Minimization with zfit

zfit is a project that aims to establish a fitting framework in Python capabale enough to perform High Energy Physics analysis. The project focuses on two aspects:
 - fast and scalable fits
 - well defined workflow
 
The first aspect is achieved by using a backend such as TensorFlow (or JAX in the future). There are dedicated tutorials available on this aspect but it is, for the beginning, not our first priority.

The second aspect concerns the workflow, which is depicted below.

![zfit workflow](images/zfit_workflow.png)

After this, other libraries such as hepstats will pick up the elements created with zfit and do further statistical inference.

For a user, the features of zfit and hepstats broadly include:
 - build complicated models in multiple dimensions allowing for composite models
 - strong support for custom created models in pure Python
 - (likelihood) minimization for unbinned (extended) fits including arbitrary constraints
 - sPlot and sWeights
 - Confidence Intervalls and significance calculation
 
For a developer of a fitting (related) library, zfit aims to establish an ecosystem with a stable API and to provide base classes that allow to build on top; the focus is on a stable core, not on a lot of content in zfit.

## What is zfit and hepstats?

To get an impression of what zfit and hepstats can do, we'll use a simple example to determine the upper limit of a signal.

In [ ]:
import hepstats
import mplhep
import numpy as np
import zfit

In [ ]:
# create some data
bounds = (0.1, 3.0)
bkg = np.random.exponential(0.5, 300)
peak = np.random.normal(1.2, 0.1, 10)
data = np.concatenate((bkg, peak))
data_np = data[(data > bounds[0]) & (data < bounds[1])]
N = data.size

mplhep.histplot(np.histogramdd(data, bins=50))

## building the likelihood: model and data

In [ ]:
obs = zfit.Space('x', limits=bounds)
data = zfit.Data.from_numpy(obs=obs, array=data_np)

# create the parameters
lambda_ = zfit.Parameter("lambda", -2.0, -4.0, -1.0)
Nsig = zfit.Parameter("Nsig", 1., -20., N)
Nbkg = zfit.Parameter("Nbkg", N, 0., N*1.1)

# create the total model as a sum of two extended pdfs
signal = zfit.pdf.Gauss(obs=obs, mu=1.2, sigma=0.1).create_extended(Nsig)
background = zfit.pdf.Exponential(obs=obs, lambda_=lambda_).create_extended(Nbkg)
model = zfit.pdf.SumPDF([signal, background])

# loss function from model and data
loss = zfit.loss.ExtendedUnbinnedNLL(model=model, data=data)

# minimization (not required, just for demonstration)
minimizer = zfit.minimize.Minuit()
result = minimizer.minimize(loss)
result.hesse()
print(result)

## Inference: setting an upper limit

In [ ]:
from hepstats.hypotests import UpperLimit
from hepstats.hypotests.calculators import AsymptoticCalculator
from hepstats.hypotests.parameters import POI, POIarray

# Use the loss and minimizer from zfit
calculator = AsymptoticCalculator(loss, minimizer, asimov_bins=100)

# define null and alternative hypothesis
poinull = POIarray(Nsig, np.linspace(0.0, 50, 40))
poialt = POI(Nsig, 0)
ul = UpperLimit(calculator, poinull, poialt)
ul.upperlimit(alpha=0.05, CLs=True)

Pretty simpel. But before we will have a deeper look into the components and better understand the libraries, we will focus on recent news in zfit: minimizers.

## Minimization: API and workflow

The workflow of zfit is designed that each stage is maximally decoupled and can be used as standalone as possible.

**You don't need to use zfit to use zfit.** But you can use the pieces that you want to.

*(this has it's limits: if we don't use zfit models in the loss, most hepstat functionality won't work: the one that relies on knowing the API of the model. But it's easy to implement your function in a zfit model.)*

### Loss, minimizer and result

In this section, we will use just some parts of zfit, namely the part that is responsible for the minimization. As it is maybe known from other minimizers, the concepts in zfit are split as:
 - **loss**: information about the function to minimize and *constraints* (not in minimizer).
 - **minimizer**: stateless, but fully configurable. Fully minimizes a function.
 - **result**: stores all information about the fit and performs simple parameter uncertainty calculations (not in the minimizer).


In [ ]:
# set everything to numpy mode (if anything else than zfit.z is used)
zfit.run.set_autograd_mode(False)
zfit.run.set_graph_mode(False)

In [ ]:
# create a problem
def func(x):
    x = np.array(x)  # make sure it's an array
    return np.sum((x - 0.1) ** 2 + x[1] ** 4)

# we need to set the errordef
func.errordef = 0.5

# initial parameters
params = [1, -3, 2, 1.4, 11]

# create our favorite minimizer
minimizer = zfit.minimize.Minuit()

# minimize
result = minimizer.minimize(func, params)

# estimate errors
result.hesse()
result.errors(method='minuit_minos')
print(result)

### Why do we need more minimizers?

There are many good minimizers (iminuit, SciPy, ...) out there. So why even consider to implement something new?

zfit merely wraps existing minimizers with the goal of a clean interface and behavior.
There are essentially two parts of a minimizer:
 - Configuration of the algorithm: this depends a lot on the chosen algorithm (trust radius, hessian approximation, ...)
 - minimization: independent of the algorithm. Needs a function, initial parameters (and more information on the function)
 
This led to two design choices:
 - every minimizer is a class and the initializer takes the configuration and has its own documentation *but knows nothing about the minimization function*
 - one method `minimize` which takes the function *but no configuration of the algorithm*
 
Therefore, the minimizers in zfit are naturally stateless and can be interchanged *without changing the rest of the script*. They are a "configuration holding class".

**Not obvious (and not the only possible) choices.**

**iminuit** is stateful: more massaging around with the function minimization and more (efficient) functionality that can use the full state information *(->custom minimizer in zfit)*
**SciPy** mixes both into one function: design choice that leads to cluttered interface as everything is mixed together ([here](https://github.com/scipy/scipy/issues/13524) or [here](https://github.com/scipy/scipy/issues/13754))

Most minimizers do not offer the combination of
 - gradients
 - arbitrary constraints
 - unified and solid termination criteria
 - (uncertainty estimation)

**This is the role that zfit minimizers fulfill**

## zfit minimizer creation

Every minimizer [can take arbitrary arguments](https://zfit.readthedocs.io/en/latest/user_api/minimize/minimizers.html#minimizers), but there are many shared ones, such as:
 - tol: tolerance for which to terminate the minimization
 - verbosity (from 0 to 10): the verbosity of the minimizer
 - gradient (if available): `False` or `'zfit'` uses internal gradients (numerically if needed), `True` or other allowed values use the minimizers own gradient function
 - criterion: convergence criterion (default EDM)
 - maxiter: approximate number of function evaluations
 - strategy: zfit strategy to deal with NaNs
 
We can now create a minimizer of our choice

In [ ]:
# create our favorite minimizer
# minimizer = zfit.minimize.IpyoptV1()
# minimizer = zfit.minimize.Minuit()
minimizer = zfit.minimize.ScipyTrustConstrV1()
# minimizer = zfit.minimize.NLoptLBFGSV1()

Done. Now we can see how to minimize the function. Therefore we need a loss.

## zfit "loss"

Minimizers accept anything that:
 - takes one positional argument, the array of values to evaluate the loss function at and returns the function value
 - has an attribute `errordef`
 
A zfit loss already fulfils this criteria.

### intermezzo: iminuit

iminuit accepts the same functions! So anything that you want to minimize with zfit also works with iminuit out-of-the-box.

A zfit loss can be called with an array of elements, where the order corresponds to the return value of `get_params()`. Let's re-use the loss we built above and try it out

In [ ]:
loss.get_params()

In [ ]:
from iminuit import Minuit

minuit = Minuit(loss, [10, 250, -2.5])
minuit.migrad()

### the minimize method

Minimize takes exactly three arguments:
 - loss: the function to minimize
 - params: information about the parameters such as starting point
 - init (optional): a `FitResult` from a previous minimization that contains additional infromation

In [ ]:
# for a more fine-grained control
params = {
    'value': [1, -3, 2],  # mandatory
    'lower': [-2, -5, -5],  # lower bound, optional
    'upper': [2, 4, 5],  # upper bound, optional
    'step_size': [0.1] * 3,  # initial step size, optional
    'name': ['a', 'b', 'c']  # names, optional
}

# or we can directly create the parameters (can also be commented)
params = [zfit.Parameter(params['name'][i], params['value'][i],
          params['lower'][i], params['upper'][i], params['step_size'][i])
          for i in range(len(params['value']))]

# minimize
result = minimizer.minimize(func, params)
print(result)

### Comparable criterion

As we've seen, we can fully mix or swap minimizers in zfit and use the same code for the minimization.

The result shows also an "edm", the Estimated Distance to Minimum, which is also the criterion used by iminuit. zfit minimizers use by default the same, which makes results comparable.

However, this criterion needs the (approximate) inverse Hessian matrix and therefore can be quite slow to compute in general, especially if the minimizer does not provide an approximation (if they do, this is used). More criteria are welcome to be added to zfit!

## FitResult

The return value is a FitResult. While there are already a few standards, such as Scipys OptimizeResult, zfits FitResult is more powerful: it also contains uncertainty calculations, both a hessian approximation as well as a profiling method ("minos").

But first, lets look at a few useful flags.

In [ ]:
result.valid  # tells if the fit is valid: converged, not at limits etc

In [ ]:
result.fmin  # minimum value

In [ ]:
result.params  # the parameters in a dict-like form (use print for a nice view)

In [ ]:
result.info  # contains (non everything is guaranteed) additional infromation including the "original" result

### Uncertainty estimation

The `FitResult` implements, for both the hessian and profiling, the exposed iminuit error methods as well as pure Python implementations (standalone, independent of iminuit).

In [ ]:
result.hesse(method='minuit_hesse')

In [ ]:
result.hesse(method='hesse_np')

In [ ]:
print(result)

Both uncertainties have been added to the result.

We can also choose a different name and a differenc cl!

In [ ]:
result.hesse(method='hesse_np', name='hesse_90cl', cl=0.9)
print(result)

We can also use the profiling methods in the same manner. Note that this returns two values: the errors in a dict as well as a new `FitResult` (in case a new minimum was found, otherwise `None`).

`errors` takes also a name and a cl argument.

In [ ]:
result.errors(method='zfit_error')
result.errors(method='minuit_minos')
print(result)

We see that both methods agree with each other!

In [ ]:
params

## intermezzo: what did actually happen?

The function that we gave to `minimize` was wrapped with a `SimpleLoss` (and the parameters converted to `zfit.Parameter` if they were not initially). We can access the loss (and minimizer) directly in the result.

In [ ]:
loss_created = result.loss
print(result.minimizer)

In [ ]:
# we can also retrieve the parameters again
params = loss_created.get_params()
print(params)

We could also do this directly to have more control.

In [ ]:
loss_direct = zfit.loss.SimpleLoss(func=func, params=params, errordef=0.5)

### Chaining minimizers with init

The third argument to `minimize` contains additional information, it has to be a `FitResult`. This encodes the state of the minimizers and allows to "chain" minimizers!

We can either give loss and params as well *or, if we want to continue the exact same minimization*, just the result. The loss and params will automatically be taken from it.

In [ ]:
result.loss.get_params()

In [ ]:
minimizer2 = zfit.minimize.Minuit(tol=1e-6)
minimizer2.minimize(result.loss, params, result)
# or as a shortcut
# minimizer2.minimize(result)

This allows to have first a more global minimizer and then a more local one.

## Creating a custom minimizer

It wouldn't be a zfit feature if there wasn't a straightforward way to build your own minimizer ;)

We could just imitate the simple interface or use the strong base class that will take most of the heavy lifting.

In [ ]:
from zfit.minimize import minimize_supports


class Pyhep2021Minimizer(zfit.minimize.BaseMinimizer):
    def __init__(self, minimizers, **kwargs):
        super().__init__(**kwargs)
        self._other_minimizers = minimizers
        
    @minimize_supports(init=True)  # when set to False, it will always be None and the initial values set correctly
    def _minimize(self, loss, params, init):
        result = init
        for minimizer in self._other_minimizers:
            print(f"minimizing with {minimizer}")
            result = minimizer.minimize(loss, params,result)
        return result

In [ ]:
custom_minimizer = Pyhep2021Minimizer([minimizer, minimizer2])

In [ ]:
custom_minimizer.minimize(func, params)

### Useful helpers

As we have seen, the minimizer `__init__` can take quite a few arguments such as criterion, strategy and more. Plus, we also don't have gradients yet.

The base class has a few helpers, most notably
 - `evaluator = self.create_evaluator()`: this creates an evaluator. It is similar to the loss but can be used
   by calling `evaluator.value(x)` or `evaluator.gradient(x)` or `evaluator.hessian(x)` and has a `maxiter_reached` attribute. The evaluator takes care of evaluating everything correctly including to apply the strategy that can push back on NaNs or custom callbacks.
 - `criterion = self.create_criterion()`: This creates an istance of the criterion. It has a method `converged(...)`, which takes a `FitResult` and tells whether it is converged. `last_value` returns the last calculated value while `calculate(...)` explicitly calculates it.
 
When called inside `_minimize`, arguments can be omitted to the two helpers. Otherwise they would need to be specified.


## Summary

zfit offers a standardized minimizer interface and a result that decouples the uncertainty estimation from the minimization process. Currently, NLopt, SciPy, iminuit, Ipyopt and TensorFlow optimizers are wrapped, but more can be added!

This is a relatively new feature, there are some rought edges:
 - maybe it complains on parameter creation that this one already exists
 - many parameters can be extremely slow (especially for minimizers without a Hessian approximation) due to the EDM criterion